In [1]:
from Logger import Logger

logger = Logger()
logger.start_session(subject="MFI", session="Evening")


The available subjects are: ['MFI', 'Valuation', 'IB']


The available sessions are: ['Morning', 'Afternoon', 'Evening']

Study session started at 23:43:58
Studying for: 00:00:00        

In [2]:
while True:
    continue

Studying for: 00:00:04        

KeyboardInterrupt: 

In [3]:
logger.end_session()

Study session ended at 23:44:04
{'day': datetime.date(2025, 1, 13), 'start_time': datetime.datetime(2025, 1, 13, 23, 43, 58, 347850), 'end_time': datetime.datetime(2025, 1, 13, 23, 44, 4, 850052), 'total_time': datetime.timedelta(seconds=6, microseconds=502202), 'session': 'Evening', 'subject': 'MFI'}
File saved at: ./Logs/study_sessions.csv


In [14]:
logger.export_cache()

File saved at: ./Logs/study_sessions.csv
Deleted: ./Logs/Cache\log_23_26_03.csv


In [6]:
import os
import pandas as pd

cache_df_names_list = os.listdir("./Logs/Cache")

cache_dfs = []

for name in cache_df_names_list:
    cache_dfs.append(pd.read_csv(f"./Logs/Cache/{name}"))

df_out = pd.concat(cache_dfs, axis=0)

df_out


,day,start_time,end_time,total_time,session,subject
0,2025-01-11,2025-01-11 23:18:59.858956,2025-01-11 23:19:14.832056,0 days 00:00:14.973100,Evening,MFI


In [6]:
import pandas as pd

df_ex = pd.read_csv("./Logs/study_sessions.csv", index_col=0)
df_ex['start_time'] = pd.to_datetime(df_ex['start_time'])
df_ex['end_time'] = pd.to_datetime(df_ex['end_time'])
df_ex['total_time'] = pd.to_timedelta(df_ex['total_time'])
#df_ex
df_ex.groupby(["day"]).agg({"total_time": "sum"})


,total_time
day,
2025-01-12,0 days 08:11:35.344571
2025-01-13,0 days 07:58:27.263957


In [8]:
df_ex.pivot_table(index="day", columns=["subject"], values="total_time", aggfunc="sum").fillna(pd.Timedelta(seconds=0))

subject,MFI,Valuation
day,,
2025-01-12,0 days 04:10:39.427286,0 days 04:00:55.917285
2025-01-13,0 days 02:35:55.740378,0 days 05:22:31.523579
